In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

import bokeh
from bokeh.palettes import RdYlGn
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, ColumnDataSource
from bokeh.plotting import figure, output_file, show
from bokeh.transform import linear_cmap
from bokeh.models.tools import HoverTool

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

CHROMEDRIVER_PATH = "/Users/paulemiledugnat/Downloads/chromedriver_upd"

In [6]:
league_name = "Ligue_1"

chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options=chrome_options, executable_path=CHROMEDRIVER_PATH)

driver.get(f"https://understat.com/league/{league_name}")

team_soup = BeautifulSoup(driver.page_source)
table_html = team_soup.find("div", {"id": "league-chemp"}).find("table")

driver.quit()

In [97]:
df_league.head(2)

,№,Team,M,W,D,L,G,GA,PTS,xG,xGA,xPTS
0,1,Lille,29,18,9,2,49,17,63,38.68-10.32,17.55+0.55,54.18-8.82
1,2,Paris Saint Germain,29,19,3,7,63,19,60,64.56+1.56,29.74+10.74,58.69-1.31


In [98]:
def process_html_ligue(html_league_table: str):

    df_league = pd.read_html(str(table_html))[0]

    for col in ["xG", "xGA", "xPTS"]:
        df_league[col] = df_league[col].str.split('[+-]', expand=True)[0].astype(float)
        df_league[f"diff_{col}"] = df_league[col[1:]] - df_league[col]

    df_league["theoretical_rank"] = df_league["xPTS"].rank(ascending=False)
    df_league["true_rank"] = df_league["PTS"].rank(ascending=False)
    
    return df_league

In [99]:
df_league.head(2)

,№,Team,M,W,D,L,G,GA,PTS,xG,xGA,xPTS,diff_xG,theoretical_rank,diff_xGA,diff_xPTS,true_rank
0,1,Lille,29,18,9,2,49,17,63,38.68,17.55,54.18,10.32,4.0,-0.55,8.82,1.0
1,2,Paris Saint Germain,29,19,3,7,63,19,60,64.56,29.74,58.69,-1.56,2.0,-10.74,1.31,2.5


In [100]:
from bokeh.layouts import row
from bokeh.models import (CategoricalTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, NumeralTickFormatter, FactorRange)
from bokeh.models.tools import HoverTool
from bokeh.palettes import RdYlGn
from bokeh.plotting import figure, output_file, show
from bokeh.plotting.figure import Figure
from bokeh.transform import linear_cmap

In [85]:
def plot_xG_league(df_xG_league: pd.DataFrame, league_name: str, year: int, mode="G") -> Figure:
    if mode == "G":
        full_mode = "Goal"
    elif mode == "GA":
        full_mode = "GoalAgainst"
    elif mode == "PTS":
        full_mode = "Points"
    else:
        raise AttributeError(f"No such mode {mode}")
        
    k_offset = 5

    plot_max = max(df_xG_league[f"x{mode}"].max() + k_offset,
                   df_xG_league[f"{mode}"].max() + k_offset)
    
    plot_min = min(df_xG_league[f"x{mode}"].min() - k_offset,
               df_xG_league[f"{mode}"].min() - k_offset)

    amplitude = max(abs(df_xG_league[f"diff_x{mode}"].min()),
                    abs(df_xG_league[f"diff_x{mode}"].max()))

    color_mapper = LinearColorMapper(
        palette=RdYlGn[9][::-1], low=-amplitude, high=amplitude)

    fig = figure(
        title=f"x{full_mode} vs. vrais {full_mode} pour {league_name}, saison {year}-{year + 1}",
        y_range=(plot_min, plot_max),
        x_range=(plot_min, plot_max),
        plot_width=900,
        plot_height=600,
    )

    fig.xaxis.axis_label = f'x{full_mode}'
    fig.yaxis.axis_label = f'{full_mode}'
    fig.xaxis.axis_label_text_font_size = "18pt"
    fig.yaxis.axis_label_text_font_size = "18pt"

    fig.line([0, plot_max], [0, plot_max], color="black",
             legend_label="Performance normale", line_width=2)

    r = fig.circle(x=f'x{mode}',
                   y=f'{mode}',
                   source=df_xG_league,
                   size=10,
                   color={'field': f'diff_x{mode}', 'transform': color_mapper})

    glyph = r.glyph
    glyph.size = 15
    glyph.fill_alpha = 1
    glyph.line_color = "black"
    glyph.line_width = 1

    fig.background_fill_color = "gray"
    fig.background_fill_alpha = 0.05

    hover = HoverTool()
    if mode == "G":
        hover.tooltips = [
            ('', '@Team'),
            ('xG', '@xG{0.2f}'),
            ('G', '@G{0.2f}'),
            ('Diff. xG vs G', '@diff_xG{0.2f}')
        ]
    elif mode == "GA":
        hover.tooltips = [
            ('', '@Team'),
            ('xGA', '@xGA{0.2f}'),
            ('GA', '@GA{0.2f}'),
            ('Diff. xGA vs GA', '@diff_xGA{0.2f}')
        ]
    elif mode == "PTS":
        hover.tooltips = [
            ('', '@Team'),
            ('xPTS', '@xPTS{0.2f}'),
            ('PTS', '@PTS{0.2f}'),
            ('Diff. xPTS vs PTS', '@diff_xPTS{0.2f}')
        ]
    else:
        raise AttributeError(f"No such mode {mode}")

    color_bar = ColorBar(color_mapper=color_mapper, width=8)

    fig.add_layout(color_bar, 'right')
    fig.add_tools(hover)
    fig.legend.location = "top_left"

    fig.toolbar.logo = None
    fig.toolbar_location = None

    show(fig)

In [88]:
plot_xG_league(df_league, league_name="Ligue_1", year=2020, mode="PTS")

In [4]:
from bokeh.io import output_file, show
from bokeh.palettes import Spectral5
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

In [8]:
import itertools

In [20]:
import config

ModuleNotFoundError: No module named 'config'

In [14]:
def get_xG_html_table(team_name: str, year: int, force_update: bool = False, stats: str = "players") -> str:
    path_name = os.path.join(
        config.CACHE_PATH, f"{team_name}_{year}_{stats}.txt")

    # try cache
    if os.path.exists(path_name) and not force_update:
        with open(path_name) as cache_text:
            table_html = cache_text.read().replace('\n', '')
        return table_html

    driver = get_driver()

    driver.get(f"https://understat.com/team/{team_name}/{year}")
    team_soup = BeautifulSoup(driver.page_source)
    table_html = str(team_soup.find(
        "div", {"id": f"team-{stats}"}).find("table"))

    driver.quit()

    with open(path_name, "w") as cache_text:
        cache_text.write(table_html)

    return table_html


In [21]:
import os

CACHE_PATH = "data_cache"

if 'DYNO' in os.environ:     # heroku env
    GOOGLE_CHROME_PATH = '/app/.apt/usr/bin/google_chrome'
    CHROMEDRIVER_PATH = '/app/.chromedriver/bin/chromedriver'
else:                        # local env
    GOOGLE_CHROME_PATH = ''
    CHROMEDRIVER_PATH = "src/chromedriver_upd"

LIST_OF_YEARS = [2020, 2019, 2018, 2017, 2016, 2015, 2014]

LIST_OF_COUNTRIES = ["<Choix d'un pays>", "France", "Espagne",
                     "Angleterre", "Allemagne", "Italie"]

CHOIX_TEAMS = ["<Choix d'un pays>"]

FRENCH_TEAMS = ["<Choix d'une équipe>", "Lille", "Lyon", "Paris_Saint_Germain",
                "Monaco", "Lens", "Metz", "Marseille",
                "Rennes", "Lorient", "Strasbourg",
                "Montpellier", "Bordeaux", "Nice", "Brest",
                "Angers", "Nantes", "Reims", "Nantes",
                "Dijon", "Nimes"]

GERMAN_TEAMS = ["<Choix d'une équipe>", 'Werder_Bremen', 'VfB_Stuttgart', 'Union_Berlin', 'Schalke_04',
                'RasenBallsport_Leipzig', 'Mainz_05', 'Hoffenheim',
                'Hertha_Berlin', 'Freiburg', 'FC_Cologne', 'Eintracht_Frankfurt',
                'Borussia_M.Gladbach', 'Borussia_Dortmund', 'Bayern_Munich',
                'Bayer_Leverkusen', 'Augsburg', 'Arminia_Bielefeld']

ENGLISH_TEAMS = ["<Choix d'une équipe>", 'Manchester_City', 'Manchester_United', 'Leicester', 'West_Ham', 'Chelsea',
                 'Liverpool', 'Everton', 'Aston_Villa', 'Tottenham', 'Leeds',
                 'Arsenal', 'Wolverhampton_Wanderers', 'Crystal_Palace',
                 'Southampton', 'Burnley', 'Brighton', 'Newcastle_United', 'Fulham',
                 'West_Bromwich_Albion', 'Sheffield_United']

SPANISH_TEAMS = ["<Choix d'une équipe>", 'Atletico_Madrid', 'Real_Madrid', 'Barcelona', 'Sevilla',
                 'Real_Sociedad', 'Villarreal', 'Real_Betis', 'Levante', 'Granada',
                 'Athletic_Club', 'Celta_Vigo', 'Valencia', 'Osasuna', 'Cadiz',
                 'Getafe', 'Alaves', 'Eibar', 'Real_Valladolid', 'Elche',
                 'SD_Huesca']

ITALIAN_TEAMS = ["<Choix d'une équipe>", 'Inter', 'AC_Milan', 'Juventus', 'Roma', 'Atalanta', 'Lazio',
                 'Napoli', 'Sassuolo', 'Verona', 'Sampdoria', 'Genoa', 'Bologna',
                 'Udinese', 'Fiorentina', 'Benevento', 'Spezia', 'Torino',
                 'Cagliari', 'Parma_Calcio_1913', 'Crotone']

COUNTRY_TEAMS = {"France": FRENCH_TEAMS,
                 "Espagne": SPANISH_TEAMS,
                 "Angleterre": ENGLISH_TEAMS,
                 "Allemagne": GERMAN_TEAMS,
                 "Italie": ITALIAN_TEAMS,
                 "<Choix d'un pays>": CHOIX_TEAMS}


In [16]:
def update_db(list_teams, list_years):
    for team, year in itertools.product(list_teams, list_years):
        get_xG_html_table(team, year, force_update=True)



In [22]:
update_db(["Lyon", "Lille"], [2020])

NameError: name 'config' is not defined

In [6]:
st.sidebar.header("Paramètres")

country_choice = st.sidebar.selectbox(
    'Quelle pays veux-tu analyser ?',
    config.LIST_OF_COUNTRIES,
    index=0)

team_choice = st.sidebar.selectbox(
    'Quelle équipe veux-tu analyser ?',
    config.COUNTRY_TEAMS[country_choice],
    index=0)

year_choice = st.sidebar.selectbox(
    'Quelle année veux-tu analyser ?',
    config.LIST_OF_YEARS,
    index=0)

st.sidebar.header("Analyses")

st.sidebar.text("")
goal_options = st.sidebar.checkbox("Montrer les buts", value=True)

st.sidebar.text("")
assist_options = st.sidebar.checkbox("Montrer les assists", value=True)

st.sidebar.text("")
situations_options = st.sidebar.checkbox("Montrer les situations", value=True)

st.sidebar.text("")
shots_quality_options = st.sidebar.checkbox(
    "Montrer la qualité des tirs", value=True)

NameError: name 'st' is not defined

In [130]:
df_stats = df_stats[df_stats["Situation"] != "Penalty"]
x = list(itertools.product(df_stats["Situation"], ["xG/Sh", "xGA/Sh"]))
counts = sum(zip(df_stats['xG/Sh'], df_stats['xGA/Sh']), ())

source = ColumnDataSource(data=dict(x=x, counts=counts))
_ = "représente la qualité des tirs vs. la qualité des tirs adverses"

p = figure(x_range=FactorRange(*x), title="Différence de xG pour (xG) et contre (xGA) par situation de jeu",
           toolbar_location=None, tools="", tooltips="@x : @counts{0.2f}")

amplitude = max(counts)
amplitude = 0.2
color_mapper = LinearColorMapper(palette=RdYlGn[11][::-1], low=0, high=amplitude)

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="black", line_width=0.5,
       fill_color={'field': "counts", 'transform': color_mapper},
       )

p.y_range.start = 0
p.xgrid.grid_line_color = None

show(p)

In [131]:
type(p)

bokeh.plotting.figure.Figure

In [116]:
x, counts

([('Open play', 'xG/Sh'),
  ('Open play', 'xGA/Sh'),
  ('From corner', 'xG/Sh'),
  ('From corner', 'xGA/Sh'),
  ('Set piece', 'xG/Sh'),
  ('Set piece', 'xGA/Sh'),
  ('Direct Freekick', 'xG/Sh'),
  ('Direct Freekick', 'xGA/Sh'),
  ('Penalty', 'xG/Sh'),
  ('Penalty', 'xGA/Sh')],
 (0.15, 0.08, 0.09, 0.07, 0.12, 0.09, 0.05, 0.07, 0.76, 0.76))

In [114]:
y = fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack
x, counts

([('Apples', '2015'),
  ('Apples', '2016'),
  ('Apples', '2017'),
  ('Pears', '2015'),
  ('Pears', '2016'),
  ('Pears', '2017'),
  ('Nectarines', '2015'),
  ('Nectarines', '2016'),
  ('Nectarines', '2017'),
  ('Plums', '2015'),
  ('Plums', '2016'),
  ('Plums', '2017'),
  ('Grapes', '2015'),
  ('Grapes', '2016'),
  ('Grapes', '2017'),
  ('Strawberries', '2015'),
  ('Strawberries', '2016'),
  ('Strawberries', '2017')],
 (2, 5, 3, 1, 3, 2, 4, 3, 4, 3, 2, 4, 2, 4, 5, 4, 6, 3))

In [96]:
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=250, title="Fruit counts by year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

KeyError: 'Situation'

In [4]:
def process_html_stat(html_table: str) -> pd.DataFrame:
    df_team = pd.read_html(html_table)[0].drop("№", axis=1)
    
    df_team["xG"] = df_team["xG"].str.split(r"\+|\-").apply(lambda x: float(x[0]))
    df_team["xGA"] = df_team["xGA"].str.split(r"\+|\-").apply(lambda x: float(x[0]))

    df_team["diff_xG"] = (df_team["G"] - df_team["xG"])
    df_team["diff_xGA"] = (df_team["GA"] - df_team["xGA"])
    
    return df_team

In [40]:
from bokeh.models import CategoricalTicker, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.models.tools import HoverTool


In [41]:
df = process_html_stat(str(table_html))
df.head(2)

,Situation,Sh,G,ShA,GA,xG,xGA,xGD,xG/Sh,xGA/Sh,diff_xG,diff_xGA
0,Open play,319,37,223,19,47.19,17.36,29.83,0.15,0.08,-10.19,1.64
1,From corner,68,2,45,1,6.01,3.09,2.92,0.09,0.07,-4.01,-2.09


from bokeh.layouts import row

In [77]:
def situation_chart(df):

    x = df['Situation']
    y = df['diff_xG']

    amplitude = df['diff_xG'].abs().max() + 1
    color_mapper = LinearColorMapper(palette=RdYlGn[11][::-1], low=-amplitude, high=amplitude)

    h_barchart = figure(
        title='Visualisation de la diff xG pour Lyon, saison 2020-2021',
        y_range=x.values,
        plot_width=1500,
        plot_height=500,
        x_range=(- amplitude, amplitude))

    h_barchart.yaxis.ticker = CategoricalTicker()
    r = h_barchart.hbar(right=y, y=x, height=0.2,
        color={'field': "right", 'transform': color_mapper},
        )

    glyph = r.glyph
    glyph.fill_alpha = 1
    glyph.line_color = "black"
    glyph.line_width = 0.2

    hover = HoverTool()
    hover.tooltips = [
        ('diff xG', '@y'),
        ('Catégorie', '@right{0.2f}'),
    ]

    h_barchart.add_tools(hover)
    h_barchart.toolbar.logo = None
    h_barchart.toolbar_location = None

    color_bar = ColorBar(color_mapper=color_mapper, width=12)
    color_bar_plot = figure(height=500, width=100, 
                            toolbar_location="right")

    color_bar_plot.add_layout(color_bar, 'right')
    color_bar_plot.toolbar.logo = None
    color_bar_plot.toolbar_location = None

    layout = row(h_barchart, color_bar_plot)

    show(layout)

In [78]:
situation_chart(df)

In [80]:
from selenium.webdriver.chrome.options import Options


In [81]:
def get_xG_html_table(team_name: str) -> str:

    driver.get(f"https://understat.com/team/{team_name}/2020")
    
    team_soup = BeautifulSoup(driver.page_source)
    table_html = team_soup.find("div", {"id": "team-players"}).find("table")
    
    driver.quit()

    return str(table_html)

In [83]:
get_xG_html_table()

'<table><thead style="opacity: 1;"><tr><th class="sort" data-num="1"><span>№</span></th><th class="sort" data-num="2" width="110"><span title="Player name">Player</span></th><th data-num="3"><span title="Position">Pos</span></th><th class="sort" data-num="4"><span title="Appearances">Apps</span></th><th class="sort" data-num="5"><span title="Minutes played">Min</span></th><th class="sort" data-num="6"><span title="Goals">G</span></th><th class="sort" data-num="8"><span title="Passes led to goal">A</span></th><th class="sort" data-num="9"><span title="Shots per 90 minutes">Sh90</span></th><th class="sort" data-num="10"><span title="Passes led to shot per 90 minutes">KP90</span></th><th class="sort" data-num="11" width="100"><span class="title-expected" title="Expected Goals">xG</span></th><th class="sort" data-num="13" width="100"><span class="title-expected" title="The sum of Expected Goals off shots from a player\'s key passes">xA</span></th><th class="sort" data-num="16"><span class=

In [8]:
def process_html(html_table: str) -> pd.DataFrame:
    df_team = pd.read_html(html_table)[0].drop("№", axis=1).iloc[:15]
    
    df_team["xG"] = df_team["xG"].str.split(r"\+|\-").apply(lambda x: float(x[0]))
    df_team["xA"] = df_team["xA"].str.split(r"\+|\-").apply(lambda x: float(x[0]))

    df_team["diff_xG"] = (df_team["G"] - df_team["xG"])
    df_team["diff_xA"] = (df_team["A"] - df_team["xA"])

    df_team = df_team[df_team["xG"] > 0.5] # select only players that could score 
    df_team = df_team.round(2)
    
    return df_team

In [4]:
def plot_xG_df(df_xG_team: pd.DataFrame, team_name: str) -> None:
    plot_max = df_xG_team["xG"].max() + 1

    amplitude = max(abs(df_xG_team["diff_xG"].min()), abs(df_xG_team["diff_xG"].max()))
    
    color_mapper = LinearColorMapper(
        palette=RdYlGn[9][::-1], low=-amplitude, high=amplitude)

    p = figure(title=f"xG vs. vrais buts à {team_name} 2020-2021")

    p.line([0, plot_max], [0, plot_max], color="black",
           legend_label="Performance normale", line_width=2)

    p.line([0, plot_max], [0, 1.2 * plot_max], color="black",
           line_dash=[4, 4], legend_label="Surperf de 20 %", line_color='green', line_width=2)

    p.line([0, plot_max], [0, 0.8 * plot_max], color="black",
           line_dash=[4, 4], legend_label="Sousperf de 20 %", line_color='red', line_width=2)

    p.line([0, plot_max], [0, 1.4 * plot_max], color="black",
           line_dash=[4, 4], line_color='green', line_width=1)

    p.line([0, plot_max], [0, 0.6 * plot_max], color="black",
           line_dash=[4, 4], line_color='red', line_width=1)


    r = p.circle(x='xG',
                 y='G',
                 source=df_xG_team,
                 size=10,
                 color={'field': 'diff_xG', 'transform': color_mapper})

    glyph = r.glyph
    glyph.size = 15
    glyph.fill_alpha = 1
    glyph.line_color = "black"
    glyph.line_width = 1

    p.background_fill_color = "gray"
    p.background_fill_alpha = 0.05

    hover = HoverTool()
    hover.tooltips = [
        ('', '@Player'),
        ('xG', '@xG{0.2f}'),
        ('G', '@G{0.2f}'),
        ('diff_xG', '@diff_xG{0.2f}')
    ]

    color_bar = ColorBar(color_mapper=color_mapper, width=8)

    p.add_layout(color_bar, 'right')
    p.add_tools(hover)
    p.legend.location = "top_left"

    show(p)

In [5]:
html_team_table = get_xG_html_table("Paris_Saint_Germain")
df_team = process_html(html_team_table)
plot_xG_df(df_team, team_name="Paris_Saint_Germain")

In [21]:
CHOIX_TEAMS = ["<Choix d'un pays>"]

FRENCH_TEAMS = ["<Choix d'une équipe>", "Lille", "Lyon", "Paris_Saint_Germain",
                "Monaco", "Lens", "Metz", "Marseille",
                "Rennes", "Lorient", "Strasbourg",
                "Montpellier", "Bordeaux", "Nice", "Brest",
                "Angers", "Nantes", "Reims", "Nantes",
                "Dijon", "Nimes"]

GERMAN_TEAMS = ["<Choix d'une équipe>", 'Werder_Bremen', 'VfB_Stuttgart', 'Union_Berlin', 'Schalke_04',
                'RasenBallsport_Leipzig', 'Mainz_05', 'Hoffenheim',
                'Hertha_Berlin', 'Freiburg', 'FC_Cologne', 'Eintracht_Frankfurt',
                'Borussia_M.Gladbach', 'Borussia_Dortmund', 'Bayern_Munich',
                'Bayer_everkusen', 'Augsburg', 'Arminia_Bielefeld']

ENGLISH_TEAMS = ["<Choix d'une équipe>", 'Manchester_City', 'Manchester_United', 'Leicester', 'West_Ham', 'Chelsea',
                 'Liverpool', 'Everton', 'Aston_Villa', 'Tottenham', 'Leeds',
                 'Arsenal', 'Wolverhampton_Wanderers', 'Crystal_Palace',
                 'Southampton', 'Burnley', 'Brighton', 'Newcastle_United', 'Fulham',
                 'West_Bromwich_Albion', 'Sheffield_United']

SPANISH_TEAMS = ["<Choix d'une équipe>", 'Atletico_Madrid', 'Real_Madrid', 'Barcelona', 'Sevilla',
                 'Real_Sociedad', 'Villarreal', 'Real_Betis', 'Levante', 'Granada',
                 'Athletic_Club', 'Celta_Vigo', 'Valencia', 'Osasuna', 'Cadiz',
                 'Getafe', 'Alaves', 'Eibar', 'Real_Valladolid', 'Elche',
                 'SD_Huesca']

ITALIAN_TEAMS = ["<Choix d'une équipe>", 'Inter', 'AC_Milan', 'Juventus', 'Roma', 'Atalanta', 'Lazio',
                 'Napoli', 'Sassuolo', 'Verona', 'Sampdoria', 'Genoa', 'Bologna',
                 'Udinese', 'Fiorentina', 'Benevento', 'Spezia', 'Torino',
                 'Cagliari', 'Parma_Calcio_1913', 'Crotone']

COUNTRY_TEAMS = {"France": FRENCH_TEAMS,
                 "Espagne": SPANISH_TEAMS,
                 "Angleterre": ENGLISH_TEAMS,
                 "Allemagne": GERMAN_TEAMS,
                 "Italie": ITALIAN_TEAMS,
                 "<Choix d'un pays>": CHOIX_TEAMS}


In [22]:
import streamlit as st

In [115]:
option = st.selectbox(
    'Quelle équipe veux-tu analyser',
    list_of_teams)
st.write('You selected:', option)

In [25]:
teams = []
[teams.extend(COUNTRY_TEAMS[x]) for x in COUNTRY_TEAMS.keys()]
teams = [x for x in teams if x != "<Choix d'une équipe>"]

[None, None, None, None, None, None]